In [22]:
import os, sys, glob
import pandas as pd
from tqdm import tqdm
li = glob.glob("./data/실거래가/연립다세대/*") # 아파트, 오피스텔, 연립다세대
li.sort()
df = pd.DataFrame()

for i in li:
    rows = pd.read_csv(i,skiprows = 15, encoding = 'cp949',low_memory=False)
    df = pd.concat([df,rows])
df.reset_index(drop=True,inplace=True)

df.index.value_counts()

0          1
715949     1
715951     1
715952     1
715953     1
          ..
357984     1
357985     1
357986     1
357987     1
1073938    1
Length: 1073939, dtype: int64

In [24]:
print(len(df)) # 오피스텔 447133 연립다세대 10733939

1073939


In [25]:
land = pd.read_csv('./data/토지임야정보/AL_11_D003_20231007.csv',encoding='cp949')

In [26]:
land_dict = {land['법정동명'].value_counts().index[i] : land['법정동코드'].value_counts().index[i] for i in tqdm(range(len(land['법정동명'].value_counts())))}

100%|██████████| 467/467 [00:10<00:00, 46.08it/s]


In [27]:
df['법정동코드'] = 0
df['법정동코드'] = [land_dict[df['시군구'].iloc[i]] for i in tqdm(range(len(df)))]

100%|██████████| 1073939/1073939 [00:04<00:00, 261277.80it/s]


In [28]:
bunji = list(df['번지'].unique())
san_check = [s for s in tqdm(bunji) if str(s).__contains__("산")]

100%|██████████| 54906/54906 [00:00<00:00, 4363500.30it/s]


In [29]:
df['산'] = 1
for i in san_check:
    change = df[df['번지'] == i]
    for i in tqdm(change.index):
        df.loc[:,'산'][i] = 2

  0%|          | 0/88 [00:00<?, ?it/s]/var/folders/nv/hhtvgb4j61l7czw1g2vm7pdc0000gn/T/ipykernel_43667/2774753082.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'산'][i] = 2
100%|██████████| 3/3 [00:00<00:00, 4755.45it/s]


In [30]:
df['산'].value_counts() 

1    1073409
2        530
Name: 산, dtype: int64

In [31]:
df[df['본번'].isna() == True]
df = df[df['본번'].isna() == False]

In [32]:
df['토지고유번호'] = [str(df['법정동코드'].iloc[i]) + str(df['산'].iloc[i]) + str(int(df['본번'].iloc[i])).zfill(4) + str(int(df['부번'].iloc[i])).zfill(4) for i in tqdm(range(len(df)))]

100%|██████████| 1073939/1073939 [00:17<00:00, 60866.82it/s]


In [33]:
df = df.astype({'토지고유번호':'int'})
print(type(df['토지고유번호'].iloc[0]))
# 데 이터 유 형 이 잘 바 뀌 었습 니다
df.dtypes

<class 'numpy.int64'>


시군구               object
번지                object
본번                 int64
부번                 int64
건물명               object
전월세구분             object
전용면적(㎡)          float64
계약년월               int64
계약일                int64
보증금(만원)           object
월세(만원)            object
층                float64
건축년도             float64
도로명               object
계약기간              object
계약구분              object
갱신요구권 사용          object
종전계약 보증금 (만원)     object
종전계약 월세 (만원)      object
법정동코드              int64
산                  int64
토지고유번호             int64
dtype: object

In [34]:
df["계약년월"].value_counts()

202203    13822
202202    13714
202204    13499
202107    13322
202201    13226
          ...  
201309     5436
201511     5368
201509     5332
201311     5147
201312     4993
Name: 계약년월, Length: 129, dtype: int64

In [35]:
df.to_csv('./data/실거래_가공/연립다세대_실거래_10년_가공.csv', encoding = 'cp949')